### --- Day 12: Hot Springs ---

Puzzle description redacted as-per Advent of Code guidelines

You may find the puzzle description at: https://adventofcode.com/2023/day/12

In [2]:
#!import ../Utils.ipynb

In [3]:
var inputLines = LoadPuzzleInput(2023, 12);
WriteLines(inputLines);

Loading puzzle file: Day12.txt
Total lines: 1000
Max line length: 32

????#?#???.??.. 9,2
?.#?????????###.?# 1,1,2,1,5,1
.???#????#?????#?#? 1,9,4
?#?.??.#?.??? 2,1,1,1
?????????#?###???.?. 1,9


In [4]:
Console.WriteLine(inputLines[0].Split(' ')[0].Length);

15


In [5]:
IEnumerable<int[]> Shuffle(int segments, int padding, int depth) {
    for (var i = 0; i <= padding; i++) 
    {
        IEnumerable<int[]> arr;
        if (depth == 0) {
            arr = [new int[segments]];
        } else {
            arr = Shuffle(segments, padding - i, depth - 1);
        }

        foreach (var a in arr) {
            a[segments - depth - 1] = i;

            yield return a;
        }
    }
}

foreach (var s in Shuffle(3, 3, 2)) {
    Console.WriteLine(string.Join(", ", s));
}

0, 0, 0
0, 0, 1
0, 0, 2
0, 0, 3
0, 1, 0
0, 1, 1
0, 1, 2
0, 2, 0
0, 2, 1
0, 3, 0
1, 0, 0
1, 0, 1
1, 0, 2
1, 1, 0
1, 1, 1
1, 2, 0
2, 0, 0
2, 0, 1
2, 1, 0
3, 0, 0


In [6]:
record PuzzleRow(string mask, int[] segments) {
    public override string ToString() => $"PuzzleRow mask = {mask}, segments = {string.Join(',', segments)}";
}

In [7]:
PuzzleRow ParsePuzzleRow(string line) {
    var maskSegments = line.Split(' ');
    var mask = maskSegments[0];
    var segments = maskSegments[1].Split(',').Select(int.Parse).ToArray();

    return new(mask, segments);
}

var testInputLine = "?###???????? 3,2,1";
Console.WriteLine(ParsePuzzleRow(testInputLine));

PuzzleRow mask = ?###????????, segments = 3,2,1


In [8]:
IEnumerable<int[]> Shuffle(PuzzleRow puzzleRow) {
    var minSpace = puzzleRow.segments.Sum() + puzzleRow.segments.Length - 1;
    var padding = puzzleRow.mask.Length - minSpace;

    return Shuffle(puzzleRow.segments.Length, padding, puzzleRow.segments.Length - 1);
}
var testInputPuzzle = ParsePuzzleRow(testInputLine);
var testShuffleResult = Shuffle(testInputPuzzle).Count();

// ?###???????? 3,2,1 - 10 arrangements
Console.WriteLine(testShuffleResult); // We should have more than 10 right now, no validity checks

35


In [9]:
var puzzleRows = inputLines.Select(ParsePuzzleRow).ToArray();
Console.WriteLine(puzzleRows.Length);

var totalPossibleShuffles = puzzleRows.Select(p => Shuffle(p).Count()).Sum();
Console.WriteLine(totalPossibleShuffles);

1000
54841


In [10]:
string RenderCombo(PuzzleRow puzzleRow, int[] padding) {
    var sb = new StringBuilder();

    var segments = puzzleRow.segments;
    var totalLength = puzzleRow.mask.Length;

    foreach (var (seg, pad) in segments.Zip(padding)) {
        for (var i = 0; i < pad; i++) {
            sb.Append('.');
        }
        for (var i = 0; i < seg; i++) {
            sb.Append('#');
        }

        if (sb.Length < totalLength) {
            sb.Append('.');
        }
    }

    while (sb.Length < totalLength) {
        sb.Append('.');
    }

    return sb.ToString();
}
var testCombos = Shuffle(testInputPuzzle).Select(pad => RenderCombo(testInputPuzzle, pad));
foreach (var combo in testCombos.Take(20)) {
    Console.WriteLine(combo);
}

###.##.#....
###.##..#...
###.##...#..
###.##....#.
###.##.....#
###..##.#...
###..##..#..
###..##...#.
###..##....#
###...##.#..
###...##..#.
###...##...#
###....##.#.
###....##..#
###.....##.#
.###.##.#...
.###.##..#..
.###.##...#.
.###.##....#
.###..##.#..


In [11]:
bool ValidMask(string combo, string mask) {
    if (combo.Length != mask.Length) {
        throw new Exception("Lengths should be equal");
    }

    return (combo.Zip(mask).All(ch => (ch.Item1 == ch.Item2 || ch.Item2 == '?')));
}

var validTestCombos = testCombos.Where(c => ValidMask(c, testInputPuzzle.mask));
foreach (var v in validTestCombos) {
    Console.WriteLine(v);
}

// ?###???????? 3,2,1 - 10 arrangements
var validCount = validTestCombos.Count();
Console.WriteLine(validCount);

.###.##.#...
.###.##..#..
.###.##...#.
.###.##....#
.###..##.#..
.###..##..#.
.###..##...#
.###...##.#.
.###...##..#
.###....##.#
10


In [12]:
// For each row, count all of the different arrangements of operational and
// broken springs that meet the given criteria. What is the sum of those counts?

var q = from puzzle in puzzleRows
        let shuffles = Shuffle(puzzle)
        from padding in shuffles
        let rendered = RenderCombo(puzzle, padding)
        where ValidMask(rendered, puzzle.mask)
        select rendered;

var part1Answer = q.Count();
Console.WriteLine(part1Answer);  

7622


In [13]:
// 7622 is correct!
Ensure(7622, part1Answer);

### --- Part Two ---

Puzzle description redacted as-per Advent of Code guidelines

You may find the puzzle description at: https://adventofcode.com/2023/day/12

In [15]:
PuzzleRow UnfoldPuzzleRow(PuzzleRow input) {
    var mask = input.mask;
    var unfoldedMask = string.Join('?', [mask, mask, mask, mask, mask]);
    var segments = Enumerable.Range(0, 5).SelectMany(i => input.segments).ToArray();

    return new(unfoldedMask, segments);
}
var testUnfoldedInputPuzzle = UnfoldPuzzleRow(testInputPuzzle);
Console.WriteLine(testUnfoldedInputPuzzle);

PuzzleRow mask = ?###??????????###??????????###??????????###??????????###????????, segments = 3,2,1,3,2,1,3,2,1,3,2,1,3,2,1


In [16]:
// This takes too long, as suspected. We'll need another way...
// var unfoldedTestShuffleResult = Shuffle(testUnfoldedInputPuzzle).Count();
// Console.WriteLine(unfoldedTestshuffleResult);

In [17]:
// Dynamic programming is most likely the way. We are performing a lot of
// repetition of the same "lower" combinations each time:

foreach (var s in Shuffle(4, 4, 3).Take(20)) {
    Console.WriteLine(string.Join(", ", s));
}

0, 0, 0, 0
0, 0, 0, 1
0, 0, 0, 2
0, 0, 0, 3
0, 0, 0, 4
0, 0, 1, 0
0, 0, 1, 1
0, 0, 1, 2
0, 0, 1, 3
0, 0, 2, 0
0, 0, 2, 1
0, 0, 2, 2
0, 0, 3, 0
0, 0, 3, 1
0, 0, 4, 0
0, 1, 0, 0
0, 1, 0, 1
0, 1, 0, 2
0, 1, 0, 3
0, 1, 1, 0


In [18]:
// We know that, if we reach the same point in the padding string, eg: 1,0 ===
// 0,1, we know that the number of valid combos is the same from that point
// onwards

In [19]:
string RenderSingle(int segmentSize, int leftPadding, int rightPadding) {
    var sb = new StringBuilder();
    for (var i = 0; i < leftPadding; i++) {
        sb.Append('.');
    }

    for (var i = 0; i < segmentSize; i++) {
        sb.Append('#');
    }

    for (var i = 0; i < rightPadding; i++) {
        sb.Append('.');
    }

    return sb.ToString();
}
Console.WriteLine(RenderSingle(2, 2, 2));


..##..


In [20]:
bool ValidMask2(string segment, string mask, int startPos) {
    var maskSegment = mask.Substring(startPos, segment.Length);

    return (segment.Zip(maskSegment).All(ch => (ch.Item1 == ch.Item2 || ch.Item2 == '?')));
}

Console.WriteLine(ValidMask2("...##..#", "...??..?", 0));
Console.WriteLine(ValidMask2("..##..#", "...??..?", 1));

True
True


In [21]:
using PreCalc = System.Collections.Generic.Dictionary<(int startPos, int padding, int depth), long>;

In [22]:
long ShuffleAndFilter(PuzzleRow puzzleRow, PreCalc preCalcs, int startPos, int padding, int depth) {
    var key = (startPos, padding, depth);
    if (preCalcs.TryGetValue(key, out var preCalcResult)) {
        return preCalcResult;
    }
    
    var segmentCount = puzzleRow.segments.Length;
    var segment = puzzleRow.segments[segmentCount - 1 - depth];

    long result = 0;
    
    for (var i = 0; i <= padding; i++) 
    {
        // Given this start pos, we can render this segment and check if it is valid
        // if so, assuming the following segments are also valid, we can add the previous sums to the total
        
        string renderedSegment;
        int rightPadding = 1;
        if (depth == 0) {
            rightPadding = puzzleRow.mask.Length - startPos - i - segment;
        }
        
        renderedSegment = RenderSingle(segment, i, rightPadding);

        var maskMatch = ValidMask2(renderedSegment, puzzleRow.mask, startPos);

        if (!maskMatch) {
            continue;
        }

        // At this point the current segment / padding is valid

        if (depth == 0) {
            result++;
        } else {
            result += ShuffleAndFilter(puzzleRow, preCalcs, startPos + segment + i + 1, padding - i, depth - 1);
        }
    }

    preCalcs[key] = result;
    return result;
}

long ShuffleAndFilter(PuzzleRow puzzleRow) {
    var minSpace = puzzleRow.segments.Sum() + puzzleRow.segments.Length - 1;
    var padding = puzzleRow.mask.Length - minSpace;

    return ShuffleAndFilter(puzzleRow, new(), 0, padding, puzzleRow.segments.Length - 1);
}

Console.WriteLine(testInputPuzzle);
Console.WriteLine(ShuffleAndFilter(testInputPuzzle))

PuzzleRow mask = ?###????????, segments = 3,2,1
10


In [23]:
var doubleCheck = puzzleRows.Select(ShuffleAndFilter).Sum();
Console.WriteLine(doubleCheck);

7622


In [24]:
// Try this one again

Console.WriteLine(testInputPuzzle);
Console.WriteLine(testUnfoldedInputPuzzle);

// ?###???????? 3,2,1 - 506250 arrangements
var unfoldedTestShuffleResult = ShuffleAndFilter(testUnfoldedInputPuzzle);
Console.WriteLine(unfoldedTestShuffleResult);

PuzzleRow mask = ?###????????, segments = 3,2,1
PuzzleRow mask = ?###??????????###??????????###??????????###??????????###????????, segments = 3,2,1,3,2,1,3,2,1,3,2,1,3,2,1
506250


In [25]:
// Unfold your condition records; what is the new sum of possible arrangement counts?
var unfoldedPuzzleRows = puzzleRows.Select(UnfoldPuzzleRow).ToArray();
var part2Answer = unfoldedPuzzleRows.Select(ShuffleAndFilter).Sum();
Console.WriteLine(part2Answer);

4964259839627


In [26]:
// 4964259839627 is correct!
Ensure(4964259839627, part2Answer);